In [ ]:
import pandas as pd
import numpy as np

file_path = "F:/KLTN/100_Customer/h100c101.csv"

# Đọc file CSV
df = pd.read_csv(file_path,delimiter=",", header=0)

# Kiểm tra số cột thực tế
print("Số lượng cột thực tế:", df.shape[1])
print("Dữ liệu mẫu:\n", df)
print()

# Xác định kho hàng (hàng đầu tiên sau tiêu đề)
depot = df.iloc[0]
print("Kho hàng:\n", depot)
print()

customers = df.iloc[1:].reset_index(drop=True)

# Chuyen doi toa do thanh numpy array
depot_location = np.array(depot[['x', 'y']].values) # Kho hang
customer_locations = customers[['x', 'y']].values # Khach hang

print("Tọa độ kho hàng:", depot_location)
print("Tọa độ khách hàng:\n", customer_locations)

# Trọng lượng hàng và giới hạn xe
demands = customers['demand'].values
truck_capacity = 1300  # Định nghĩa tải trọng xe

# Giới hạn thời gian phục vụ
time_windows = customers[['open', 'close']].values
service_times = 5
arrival_times = customers['time'].values

# Vận tốc xe (km/phút)
truck_speed = 50 / 60.0

# Hiển thị thông tin cơ bản
print(f"Number of Customers: {len(customers)}")
print(f"Truck Capacity: {truck_capacity}")
print(f"Speed of Trucks: {truck_speed} km/m")
print(f"Arrival Times: {arrival_times}")


Số lượng cột thực tế: 6
Dữ liệu mẫu:
              x          y  demand  open  close        time
0    33.333333  41.666667       0     0   1236    0.000000
1    37.500000  58.333333      30   825    870    0.000000
2    35.000000  54.166667      10    15     67    0.000000
3    18.333333  62.500000      30    30     92    0.000000
4    25.000000  41.666667      10    10     73    0.000000
..         ...        ...     ...   ...    ...         ...
96   37.500000  25.000000      10   734    777  605.122934
97   20.833333  29.166667      10   912    969  629.292820
98   70.833333  20.833333      10   769    820  682.379189
99   37.500000  54.166667      20   997   1068  907.638079
100  25.000000  29.166667      10  1054   1127  911.272979

[101 rows x 6 columns]

Kho hàng:
 x           33.333333
y           41.666667
demand       0.000000
open         0.000000
close     1236.000000
time         0.000000
Name: 0, dtype: float64

Tọa độ kho hàng: [33.33333333 41.66666667]
Tọa độ khách hàng:

In [2]:
import numpy as np

class Vehicle:
    def __init__(self, vehicle_id, capacity, speed, depot_location):
        self.vehicle_id = vehicle_id
        self.capacity = capacity  # Sức chứa tối đa
        self.speed = speed  # Vận tốc di chuyển (km/phút)
        self.current_location = tuple(depot_location)  # Vị trí hiện tại (bắt đầu tại kho)
        self.route = [self.current_location]  # Lộ trình xe (bắt đầu từ kho)
        self.load = 0  # Khối lượng hàng đang chở
        self.time = 0  # Thời gian hiện tại của xe
    
    def can_serve(self, demand, time_window):
        """ Kiểm tra xem xe có thể phục vụ khách hàng không """
        return (self.load + demand <= self.capacity) and (self.time <= time_window[1])
    
    def serve_customer(self, customer_index, customer_locations, demands, service_times):
        """ Phục vụ khách hàng và cập nhật trạng thái xe """
        customer_x, customer_y = tuple(customer_locations[customer_index])
        distance = np.linalg.norm(np.array([customer_x, customer_y]) - np.array(self.current_location))
        travel_time = distance / self.speed  # Thời gian di chuyển
        
        # Cập nhật trạng thái xe
        self.current_location = (customer_x, customer_y)
        self.route.append(self.current_location)

        # Kiểm tra tải trọng trước khi cập nhật
        if self.load + demands[customer_index] <= self.capacity:
            self.load += demands[customer_index]
        
        self.time += travel_time + service_times[customer_index]  # Cộng thời gian phục vụ


class Environment:
    def __init__(self, num_vehicles, truck_capacity, truck_speed, depot_location, customer_locations, demands, time_windows, service_times):
        self.vehicles = [Vehicle(i, truck_capacity, truck_speed, depot_location) for i in range(num_vehicles)]
        self.unvisited_customers = list(range(len(customer_locations)))  # Danh sách khách hàng chưa phục vụ
        self.customer_locations = customer_locations
        self.demands = demands
        self.time_windows = time_windows
        self.service_times = service_times
    
    def assign_vehicle(self):
        """ Phân công xe tải dựa trên xe gần nhất có thể phục vụ """
        while self.unvisited_customers:
            for vehicle in self.vehicles:
                if not self.unvisited_customers:
                    break  # Thoát nếu không còn khách hàng

                best_customer = None
                min_distance = float('inf')

                for customer_index in self.unvisited_customers:
                    if vehicle.can_serve(self.demands[customer_index], self.time_windows[customer_index]):
                        # Tính khoảng cách đến khách hàng
                        customer_x, customer_y = tuple(self.customer_locations[customer_index])
                        distance = np.linalg.norm(np.array([customer_x, customer_y]) - np.array(vehicle.current_location))

                        # Chọn khách hàng gần nhất
                        if distance < min_distance:
                            min_distance = distance
                            best_customer = customer_index

                if best_customer is not None:
                    vehicle.serve_customer(best_customer, self.customer_locations, self.demands, self.service_times)
                    self.unvisited_customers.remove(best_customer)

    def get_routes(self):
        """ Lấy tuyến đường của từng xe """
        return [vehicle.route for vehicle in self.vehicles]


In [3]:
import gym
import numpy as np
from gym import spaces

class DVRPEnv(gym.Env):
    def __init__(self, num_customers, truck_capacity, depot_location, customer_locations, demands, time_windows, service_times):
        super(DVRPEnv, self).__init__()

        self.num_customers = num_customers
        self.truck_capacity = truck_capacity
        self.depot_location = depot_location
        self.customer_locations = customer_locations
        self.demands = demands
        self.time_windows = time_windows
        self.service_times = service_times

        # Không gian hành động: Chọn khách hàng tiếp theo để phục vụ (hoặc quay về kho)
        self.action_space = spaces.Discrete(self.num_customers + 1) 

        # Không gian trạng thái: Bao gồm tọa độ xe tải, tải trọng còn lại và thời gian hiện tại
        self.observation_space = spaces.Box(low=0, high=1000, shape=(self.num_customers + 3,), dtype=np.float32)

        # Khởi tạo trạng thái
        self.reset()

    def reset(self):
        """ Đặt lại môi trường """
        self.vehicle_location = self.depot_location
        self.remaining_capacity = self.truck_capacity
        self.current_time = 0
        self.served_customers = set()

        return self._get_observation()

    def step(self, action):
        """ Thực hiện hành động (xe tải phục vụ khách hàng hoặc quay về kho) """
        if action == self.num_customers:  # Quay về kho
            self.vehicle_location = self.depot_location
            self.remaining_capacity = self.truck_capacity
            reward = -5  # Phạt nhẹ khi về kho sớm
        else:
            customer_index = action
            if customer_index in self.served_customers or self.remaining_capacity < self.demands[customer_index]:
                reward = -10  # Phạt nếu chọn sai khách hàng
            else:
                distance = np.linalg.norm(self.customer_locations[customer_index] - np.array(self.vehicle_location))
                travel_time = distance / 50  # Tốc độ = 50 km/h

                self.current_time += travel_time + self.service_times[customer_index]
                self.vehicle_location = self.customer_locations[customer_index]
                self.remaining_capacity -= self.demands[customer_index]
                self.served_customers.add(customer_index)

                reward = -distance  # Phạt theo khoảng cách để tối ưu lộ trình ngắn nhất

        done = len(self.served_customers) == self.num_customers
        return self._get_observation(), reward, done, {}

    def _get_observation(self):
        """ Lấy trạng thái hiện tại """
        obs = np.concatenate([
            self.vehicle_location, 
            [self.remaining_capacity], 
            [self.current_time], 
            np.array([1 if i in self.served_customers else 0 for i in range(self.num_customers)])
        ])
        return obs


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class AttentionLayer(nn.Module):
    """ Lớp Attention giúp mô hình tập trung vào khách hàng quan trọng """
    def __init__(self, input_dim):
        super(AttentionLayer, self).__init__()
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        attention_weights = torch.softmax(torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(q.size(-1)), dim=-1)
        return torch.matmul(attention_weights, v)

class MARDAM(nn.Module):
    """ Mô hình Multi-Agent Deep Attention Mechanism """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MARDAM, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.attention = AttentionLayer(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.attention(x)
        x = self.fc2(x)
        return torch.softmax(x, dim=-1)  # Xác suất chọn khách hàng


In [8]:
# Khởi tạo môi trường
env = DVRPEnv(num_customers=10, truck_capacity=1300, depot_location=np.array([50, 50]),
              customer_locations=np.random.rand(10, 2) * 100, 
              demands=np.random.randint(50, 200, size=10), 
              time_windows=np.random.randint(0, 500, size=(10, 2)), 
              service_times=np.random.randint(5, 30, size=10))

# Khởi tạo mô hình
# model = MARDAM(input_dim=env.observation_space.shape[0], hidden_dim=128, output_dim=env.action_space.n)
model = MARDAM(input_dim=14, hidden_dim=128, output_dim=env.action_space.n)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Vòng lặp huấn luyện
num_episodes = 500
for episode in range(num_episodes):
    state = torch.tensor(env.reset(), dtype=torch.float32).unsqueeze(0)
    done = False
    total_reward = 0

    while not done:
        # print(state.shape)
        action_probs = model(state)
        action = torch.multinomial(action_probs, 1).item()
        next_state, reward, done, _ = env.step(action)

        loss = loss_fn(action_probs, torch.tensor([action], dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0)
        total_reward += reward

    if episode % 50 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}")


KeyboardInterrupt: 